In [1]:
import time
import torch
import numpy as np
from mmpretrain import init_model

/home/aribra/.conda/envs/openmmlab/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Original model

In [2]:
o_model_weights = "/home/exdata/istinye/swin_t_training/swin_t_tiny_in200/best_accuracy_top1_epoch_27.pth"
o_cfg = "/home/exdata/istinye/swin_t_training/swin_t_tiny_in200/metrics_original/swin-tiny_1xb256_in1k_224_1k.py"
o_model = init_model(o_cfg, o_model_weights)


01/23 14:16:03 - mmengine - INFO - Because batch augmentations are enabled, the data preprocessor automatically enables the `to_onehot` option to generate one-hot format labels.
Loads checkpoint by local backend from path: /home/exdata/istinye/swin_t_training/swin_t_tiny_in200/best_accuracy_top1_epoch_27.pth


#### Original latency

In [3]:
N = 300
durations = []
for i in range(N):
    x = torch.randn(1, 3, 224, 224)

    torch.cuda.synchronize()
    start_time = time.time()
    o_model(x)
    torch.cuda.synchronize()
    end_time = time.time()

    latency = end_time - start_time
    
    durations.append(latency)

durations = np.array(durations)

In [4]:
mean_lat_o = durations.mean()
min_lat_o = durations.min()
max_lat_o = durations.max()

fps_o = 1 / mean_lat_o
print("mean_latency = ", mean_lat_o)
print("fps = ", 1 / mean_lat_o)

mean_latency =  0.07038877805074056
fps =  14.206810058261539


In [5]:
# del o_model

### Pruned model

In [6]:
p_model_weights = "/home/exdata/istinye/swin_t_training/pruned_swin_t_tiny_in200_cont/best_accuracy_top1_epoch_32.pth"
pruned_swin_t_model_flat = "/home/exdata/istinye/swin_t_training/swin_t_backbone_Pruned_25.pth"

p_model = torch.load(pruned_swin_t_model_flat)
p_model = p_model.eval()

p_weights = torch.load(p_model_weights)

p_model.load_state_dict(p_weights['state_dict'])

<All keys matched successfully>

#### Pruned latency

In [7]:
N = 300
durations = []
for i in range(N):
    x = torch.randn(1, 3, 224, 224)

    torch.cuda.synchronize()
    start_time = time.time()
    p_model(x)
    torch.cuda.synchronize()
    end_time = time.time()

    latency = end_time - start_time
    
    durations.append(latency)

durations = np.array(durations)

In [8]:
mean_lat_p = durations.mean()
min_lat_p = durations.min()
max_lat_p = durations.max()

fps_p = 1 / mean_lat_p
print("mean_latency = ", mean_lat_p)
print("fps = ", 1 / mean_lat_p)

mean_latency =  0.05617416381835937
fps =  17.801778113396153


### Latency diff

In [9]:
diff = 1 - (fps_o / fps_p) 
diff_prcnt = diff * 100

print("diff_prcnt % = ", diff_prcnt)

diff_prcnt % =  25.548780019469884
